In [1]:
!pip install japanize-matplotlib

In [30]:
import os
import re
from glob import glob
from pathlib import Path
import math
import random
from tqdm import tqdm

import pandas as pd
import numpy as np
import cvxopt

import matplotlib.pyplot as plt
import matplotlib.pylab as plab
import japanize_matplotlib
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')

from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

#最大表示列数の指定
pd.set_option('display.max_columns', 50)
#最大表示行数の指定
pd.set_option('display.max_rows', 100)

def seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
seed(1)

In [10]:
data_path = Path('../data')

In [11]:
def sentence2words(sentences):
    """
    文章を単語に分割
    args:
        sentence: 
            文章（文字列）
    """
    # 句読点
    punc = re.compile(r'[\[,\],-.?!,:;()"|0-9]')

    # 文章を小文字（lower）に変換し、スペースで分割（split）し、
    # 句読点を取り除き（punc.sub）、語wordsを取り出す
    words = [[punc.sub("",word) for word in sentence.lower().split()] for sentence in sentences] 

    # 空の要素を削除
    if words.count(""):
        words.pop(words.index(""))

    return words

In [12]:
# データの生成
dataType = 1
# データの選択
if dataType==1:   # amazonのレビュー
    fName = 'amazon_cells_labelled.txt'
elif dataType==2: # yelpのレビュー
    fName = 'yelp_labelled.txt'
elif dataType==3: # imdbのレビュー
    fName = 'imdb_labelled.txt'

# csvファイルの読み込み
data = pd.read_csv(os.path.join(data_path, 'sentiment labelled sentences', fName),'\t',header=None)

# データのインデックスをランダムにシャッフル
dNum = len(data)
randInds = np.random.permutation(dNum)

# 文章データ
sentences = data[0][randInds]
Y = data[1][randInds]
X = sentence2words(sentences)

In [15]:
# データを訓練と評価用に分割
dtrNum = int(len(X)*0.9)    # 学習データ数

# 訓練データ 全体の90%
X_train = X[:dtrNum]
y_train = Y[:dtrNum]
# 評価データ
X_test = X[dtrNum:]
y_test = Y[dtrNum:]

In [16]:
class naiveBayes():
    """naiveBayes"""
    def __init__(self,X,Y,priors):
        """
        args:
            x: 
                入力データ（データ数×次元数のnumpy.ndarray）
            y:
                出力データ（データ数×1のnumpy.ndarray）
            priors:
                事前確率（1×カテゴリ数のnumpy.ndarray）
        """
        # 学習データの設定
        self.X = X
        self.Y = Y

        # 重複なしの単語一覧の作成
        self.wordDict = list(np.unique(np.concatenate(self.X)))

        # 各文章の単語の出現回数
        self.wordNums = self.countWords(self.X)
        
        # 事前確率の設定
        self.priors = priors

    def countWords(self,X):
        """
        各文章中の単語の出現回数をカウント
        args:
            X:
                各文章の単語リスト（データ数のリスト）
        """
        # 各文章中の単語の出現回数をカウント
        cntWordsAll = []
        for words in X:
            cntWords = np.zeros(len(self.wordDict))

            for word in words:
                cnt = self.wordDict.count(word)

                if cnt:
                    cntWords[self.wordDict.index(word)] += cnt

            cntWordsAll.append(cntWords)

        return np.array(cntWordsAll)

    def train(self):
        """
        単語の尤度の計算
        """
        # カテゴリごとの単語の出現回数
        wordNumsCat = [np.sum(self.wordNums[self.Y==y],axis=0) for y in np.unique(self.Y)]
        wordNumsCat = np.array(wordNumsCat)

        # 単語の尤度p(x|y)（カテゴリごと単語の出現割合）の計算
        self.wordL = wordNumsCat/np.sum(wordNumsCat,axis=1,keepdims=True)

    def predict(self,X):
        """
        文章の事後確率の計算
        args:
            X: 
                各文章の単語リスト（データ数のリスト）
        """
        # 文章を単語に分割
        wordNums = self.countWords(X)

        # 文章の尤度計算
        sentenceL = [np.prod(self.wordL[ind]**wordNums,axis=1) for ind in range(len(np.unique(self.Y)))]
        sentenceL = np.array(sentenceL)
        
        # 事後確率の計算
        sentenceP = sentenceL.T * self.priors

        # 予測
        predict = np.argmax(sentenceP,axis=1)

        return predict

    def accuracy(self,X,Y):
        """
        正解率の計算
        args:
            X: 
                入力データ（データ数×次元数のnumpy.ndarray）
            Y:
                出力データ（データ数×１のnumpy.ndarray）
        """
        return np.sum(self.predict(X) - Y.T==0)/len(Y)

    def writeResult2CSV(self,X,Y,fName="../results/sentimental_results.csv"):
        """
        予測結果のcsvファイルへの書き込み
        args:
            X: 
                文章データ（データ数のリスト）
            Y:
                真値（データ数×1のnumpy.ndarray）
            fName：
                csvファイルの保存先（文字列）
        """
        P = self.predict(X)
        
        # データフレームの作成
        df = pd.DataFrame(np.array([Y,P,X]).T,columns=['gt','predict','sentence'],index=np.arange(len(X)))
        
        # csvファイルに書き出し
        df.to_csv(fName,index=False)

In [19]:
# ナイーブベイズの学習
# 事前確率の設定
priors = np.array([[0.5,0.5]])

myModel = naiveBayes(X_train, y_train, priors)
myModel.train()

In [21]:
# ナイーブベイズの評価
print(f"学習データの正解率:{np.round(myModel.accuracy(X_train, y_train),decimals=2)}")
print(f"評価データの正解率:{np.round(myModel.accuracy(X_test, y_test),decimals=2)}")

学習データの正解率:0.99
評価データの正解率:0.7


In [24]:
# 予測結果のCSVファイルへの出力
myModel.writeResult2CSV(X_train, y_train,fName=f"../results/naiveBayes_result_train_{dataType}.csv")
myModel.writeResult2CSV(X_test, y_test,fName=f"../results/naiveBayes_result_test_{dataType}.csv")

In [33]:
train_sub_df = pd.read_csv(f'../results/naiveBayes_result_train_{dataType}.csv')
train_sub_df

,gt,predict,sentence
0,0,0,"['the', 'holster', 'that', 'arrived', 'did', '..."
1,0,0,"[""doesn't"", 'do', 'the', 'job']"
2,1,1,"['my', 'colleague', '&', 'i', 'now', 'get', 'g..."
3,1,1,"['love', 'this', 'phone']"
4,0,0,"['worst', 'customer', 'service', 'ever']"
...,...,...,...
895,0,0,"['the', 'construction', 'of', 'the', 'headsets..."
896,0,0,['disappointing']
897,1,1,"['no', 'ear', 'loop', 'needed', ""it's"", 'tiny'..."
898,1,1,"['seller', 'shipped', 'quickly', 'and', 'much'..."


In [31]:
test_sub_df = pd.read_csv(f'../results/naiveBayes_result_test_{dataType}.csv')
test_sub_df

,gt,predict,sentence
0,0,0,"['att', 'is', 'not', 'clear', 'sound', 'is', '..."
1,0,0,"['poorly', 'contstruct', 'hinge']"
2,0,1,"['when', 'i', 'placed', 'my', 'treo', 'into', ..."
3,0,0,"[""doesn't"", 'work', 'at', 'all', 'i', 'bougth'..."
4,0,0,"['what', 'possesed', 'me', 'to', 'get', 'this'..."
5,1,1,"['the', 'headset', 'fulfills', 'my', 'requirem..."
6,1,0,"['this', 'phone', 'is', 'very', 'fast', 'with'..."
7,0,0,"['the', 'only', 'thing', 'that', 'i', 'think',..."
8,1,1,"['you', 'can', 'even', 'take', 'self', 'portra..."
9,1,1,"['comfort', 'for', 'our', 'whole', 'family']"
